In [2]:
import argparse
import os
import numpy as np
import math
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

In [3]:
os.makedirs("images", exist_ok=True)

In [4]:
n_epochs = 200
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 4
latent_dim = 100
img_size = 28
channels = 1
n_critic = 5
clip_value = 0.01
sample_interval = 400
corrupt_fraction = 0.1

In [5]:
img_shape = (channels, img_size, img_size)
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [8]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

In [12]:
# Configure data loader
os.makedirs("data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [10]:
def corrupt_data_loader():
    for batch in dataloader:
        batch = next(iter(dataloader1))[0]
        bathh_alt = next(iter(dataloader2))[0]
        for i in range(len(batch)):
            if torch.rand(1) <= corrupt_fraction:
                batch[i,0,:,:] = batch_alt[i,0,:,:]
        yield batch

In [32]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

In [33]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [40]:
# ----------
#  Training
# ----------

batches_done = 0
for epoch in range(n_epochs):
    for i, imgs in enumerate(corrupt_data_loader()):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        delta = 0.1
        d_loss = -real_validity.mean() + (1-delta)*fake_validity.mean() + lambda_gp * gradient_penalty + delta*real_validity.max()

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            
            if batches_done % sample_interval == 0:
                print('saving to: ' + "images_clean_robust/%d.png" % batches_done)
                save_image(fake_imgs.data[:25], "images_clean_robust/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += n_critic


[Epoch 0/200] [Batch 0/938] [D loss: -1.388977] [G loss: -2.925651]
saving to: images_clean_robust/0.png
[Epoch 0/200] [Batch 5/938] [D loss: -1.149732] [G loss: -3.193682]
[Epoch 0/200] [Batch 10/938] [D loss: -1.365308] [G loss: -2.754965]
[Epoch 0/200] [Batch 15/938] [D loss: -1.230046] [G loss: -2.495512]
[Epoch 0/200] [Batch 20/938] [D loss: -1.097627] [G loss: -2.346137]
[Epoch 0/200] [Batch 25/938] [D loss: -1.307289] [G loss: -1.532231]
[Epoch 0/200] [Batch 30/938] [D loss: -1.252381] [G loss: -2.510598]
[Epoch 0/200] [Batch 35/938] [D loss: -1.280993] [G loss: -3.034298]
[Epoch 0/200] [Batch 40/938] [D loss: -1.021985] [G loss: -2.990641]
[Epoch 0/200] [Batch 45/938] [D loss: -0.972810] [G loss: -2.171997]
[Epoch 0/200] [Batch 50/938] [D loss: -1.415833] [G loss: -2.252896]
[Epoch 0/200] [Batch 55/938] [D loss: -0.943494] [G loss: -3.018465]
[Epoch 0/200] [Batch 60/938] [D loss: -1.391762] [G loss: -2.943483]
[Epoch 0/200] [Batch 65/938] [D loss: -1.487581] [G loss: -3.808319]

[Epoch 0/200] [Batch 590/938] [D loss: -0.942234] [G loss: -2.765219]
[Epoch 0/200] [Batch 595/938] [D loss: -1.294332] [G loss: -3.164032]
[Epoch 0/200] [Batch 600/938] [D loss: -1.306782] [G loss: -3.352089]
[Epoch 0/200] [Batch 605/938] [D loss: -1.304320] [G loss: -3.011860]
[Epoch 0/200] [Batch 610/938] [D loss: -1.126148] [G loss: -2.824770]
[Epoch 0/200] [Batch 615/938] [D loss: -1.318487] [G loss: -2.341235]
[Epoch 0/200] [Batch 620/938] [D loss: -1.142142] [G loss: -2.545796]
[Epoch 0/200] [Batch 625/938] [D loss: -1.079961] [G loss: -2.167230]
[Epoch 0/200] [Batch 630/938] [D loss: -1.227189] [G loss: -2.463676]
[Epoch 0/200] [Batch 635/938] [D loss: -1.239109] [G loss: -2.731021]
[Epoch 0/200] [Batch 640/938] [D loss: -0.828330] [G loss: -3.332231]
[Epoch 0/200] [Batch 645/938] [D loss: -0.829028] [G loss: -2.820174]
[Epoch 0/200] [Batch 650/938] [D loss: -1.256100] [G loss: -2.189918]
[Epoch 0/200] [Batch 655/938] [D loss: -1.426753] [G loss: -3.137706]
[Epoch 0/200] [Batch

[Epoch 1/200] [Batch 235/938] [D loss: -1.073390] [G loss: -2.894618]
[Epoch 1/200] [Batch 240/938] [D loss: -1.134192] [G loss: -2.436667]
[Epoch 1/200] [Batch 245/938] [D loss: -0.970202] [G loss: -2.608225]
[Epoch 1/200] [Batch 250/938] [D loss: -1.300024] [G loss: -2.724034]
[Epoch 1/200] [Batch 255/938] [D loss: -1.276147] [G loss: -2.852927]
[Epoch 1/200] [Batch 260/938] [D loss: -1.509012] [G loss: -2.488357]
saving to: images_clean_robust/1200.png
[Epoch 1/200] [Batch 265/938] [D loss: -1.199836] [G loss: -2.575501]
[Epoch 1/200] [Batch 270/938] [D loss: -0.967859] [G loss: -3.008393]
[Epoch 1/200] [Batch 275/938] [D loss: -1.044696] [G loss: -3.384002]
[Epoch 1/200] [Batch 280/938] [D loss: -1.132076] [G loss: -3.647524]
[Epoch 1/200] [Batch 285/938] [D loss: -1.375877] [G loss: -4.001600]
[Epoch 1/200] [Batch 290/938] [D loss: -1.142603] [G loss: -4.429551]
[Epoch 1/200] [Batch 295/938] [D loss: -1.085615] [G loss: -3.694781]
[Epoch 1/200] [Batch 300/938] [D loss: -1.292473] 

[Epoch 1/200] [Batch 820/938] [D loss: -1.038302] [G loss: -3.856901]
[Epoch 1/200] [Batch 825/938] [D loss: -1.193614] [G loss: -3.470407]
[Epoch 1/200] [Batch 830/938] [D loss: -1.196862] [G loss: -3.083724]
[Epoch 1/200] [Batch 835/938] [D loss: -1.273577] [G loss: -3.032043]
[Epoch 1/200] [Batch 840/938] [D loss: -1.259211] [G loss: -3.514489]
[Epoch 1/200] [Batch 845/938] [D loss: -1.396445] [G loss: -3.596902]
[Epoch 1/200] [Batch 850/938] [D loss: -1.358672] [G loss: -3.648360]
[Epoch 1/200] [Batch 855/938] [D loss: -1.232859] [G loss: -3.325750]
[Epoch 1/200] [Batch 860/938] [D loss: -0.826935] [G loss: -3.699830]
[Epoch 1/200] [Batch 865/938] [D loss: -1.154892] [G loss: -3.179149]
[Epoch 1/200] [Batch 870/938] [D loss: -1.121421] [G loss: -3.088389]
[Epoch 1/200] [Batch 875/938] [D loss: -1.177428] [G loss: -3.079345]
[Epoch 1/200] [Batch 880/938] [D loss: -1.379534] [G loss: -3.108239]
[Epoch 1/200] [Batch 885/938] [D loss: -0.910213] [G loss: -3.671301]
[Epoch 1/200] [Batch

[Epoch 2/200] [Batch 465/938] [D loss: -1.056485] [G loss: -2.933237]
[Epoch 2/200] [Batch 470/938] [D loss: -1.302812] [G loss: -3.551535]
[Epoch 2/200] [Batch 475/938] [D loss: -0.948149] [G loss: -3.082820]
[Epoch 2/200] [Batch 480/938] [D loss: -1.289634] [G loss: -2.747166]
[Epoch 2/200] [Batch 485/938] [D loss: -1.026527] [G loss: -2.754179]
[Epoch 2/200] [Batch 490/938] [D loss: -1.149142] [G loss: -2.771574]
[Epoch 2/200] [Batch 495/938] [D loss: -1.351752] [G loss: -3.086258]
[Epoch 2/200] [Batch 500/938] [D loss: -0.864263] [G loss: -3.438293]
[Epoch 2/200] [Batch 505/938] [D loss: -1.039705] [G loss: -3.707678]
[Epoch 2/200] [Batch 510/938] [D loss: -0.869620] [G loss: -2.941341]
[Epoch 2/200] [Batch 515/938] [D loss: -1.459090] [G loss: -3.204406]
[Epoch 2/200] [Batch 520/938] [D loss: -1.109011] [G loss: -2.299186]
saving to: images_clean_robust/2400.png
[Epoch 2/200] [Batch 525/938] [D loss: -0.987268] [G loss: -2.467393]
[Epoch 2/200] [Batch 530/938] [D loss: -1.239281] 

...

[Epoch 18/200] [Batch 575/938] [D loss: -0.886741] [G loss: -2.782974]
[Epoch 18/200] [Batch 580/938] [D loss: -0.845964] [G loss: -2.616491]
[Epoch 18/200] [Batch 585/938] [D loss: -0.955951] [G loss: -2.800120]
[Epoch 18/200] [Batch 590/938] [D loss: -0.802782] [G loss: -2.930382]
[Epoch 18/200] [Batch 595/938] [D loss: -0.720143] [G loss: -2.931433]
[Epoch 18/200] [Batch 600/938] [D loss: -0.905154] [G loss: -3.547768]
[Epoch 18/200] [Batch 605/938] [D loss: -0.362847] [G loss: -3.894964]
[Epoch 18/200] [Batch 610/938] [D loss: -0.736377] [G loss: -3.817375]
[Epoch 18/200] [Batch 615/938] [D loss: -0.796142] [G loss: -2.903586]
[Epoch 18/200] [Batch 620/938] [D loss: -0.954235] [G loss: -2.725316]
[Epoch 18/200] [Batch 625/938] [D loss: -0.759487] [G loss: -3.104297]
[Epoch 18/200] [Batch 630/938] [D loss: -0.784540] [G loss: -3.284303]
[Epoch 18/200] [Batch 635/938] [D loss: -0.781276] [G loss: -3.589968]
[Epoch 18/200] [Batch 640/938] [D loss: -0.709142] [G loss: -3.275913]
[Epoch

[Epoch 19/200] [Batch 215/938] [D loss: -0.819956] [G loss: -2.543816]
[Epoch 19/200] [Batch 220/938] [D loss: -1.027913] [G loss: -2.257360]
[Epoch 19/200] [Batch 225/938] [D loss: -0.683302] [G loss: -2.931751]
[Epoch 19/200] [Batch 230/938] [D loss: -0.637188] [G loss: -2.819049]
[Epoch 19/200] [Batch 235/938] [D loss: -0.891368] [G loss: -2.349366]
[Epoch 19/200] [Batch 240/938] [D loss: -0.780177] [G loss: -2.868077]
[Epoch 19/200] [Batch 245/938] [D loss: -0.860339] [G loss: -2.870437]
[Epoch 19/200] [Batch 250/938] [D loss: -0.814947] [G loss: -2.903892]
[Epoch 19/200] [Batch 255/938] [D loss: -1.094450] [G loss: -3.111975]
[Epoch 19/200] [Batch 260/938] [D loss: -1.187594] [G loss: -3.364749]
[Epoch 19/200] [Batch 265/938] [D loss: -0.549775] [G loss: -3.436405]
[Epoch 19/200] [Batch 270/938] [D loss: -1.108464] [G loss: -3.290543]
[Epoch 19/200] [Batch 275/938] [D loss: -0.689598] [G loss: -3.034146]
[Epoch 19/200] [Batch 280/938] [D loss: -0.714723] [G loss: -2.558644]
[Epoch

[Epoch 19/200] [Batch 790/938] [D loss: -0.902263] [G loss: -2.861940]
[Epoch 19/200] [Batch 795/938] [D loss: -0.883268] [G loss: -2.496146]
[Epoch 19/200] [Batch 800/938] [D loss: -0.850364] [G loss: -2.540825]
[Epoch 19/200] [Batch 805/938] [D loss: -0.936472] [G loss: -3.277882]
[Epoch 19/200] [Batch 810/938] [D loss: -0.810669] [G loss: -3.274672]
[Epoch 19/200] [Batch 815/938] [D loss: -0.935238] [G loss: -3.412571]
[Epoch 19/200] [Batch 820/938] [D loss: -0.878779] [G loss: -2.646140]
[Epoch 19/200] [Batch 825/938] [D loss: -1.095833] [G loss: -2.990215]
[Epoch 19/200] [Batch 830/938] [D loss: -0.691350] [G loss: -2.956133]
[Epoch 19/200] [Batch 835/938] [D loss: -0.610135] [G loss: -2.269878]
[Epoch 19/200] [Batch 840/938] [D loss: -0.526703] [G loss: -2.476076]
[Epoch 19/200] [Batch 845/938] [D loss: -0.337610] [G loss: -2.754605]
[Epoch 19/200] [Batch 850/938] [D loss: -0.781147] [G loss: -2.392351]
[Epoch 19/200] [Batch 855/938] [D loss: -0.628204] [G loss: -2.640891]
[Epoch

[Epoch 20/200] [Batch 425/938] [D loss: -0.765639] [G loss: -2.719605]
[Epoch 20/200] [Batch 430/938] [D loss: -0.701905] [G loss: -2.327885]
[Epoch 20/200] [Batch 435/938] [D loss: -1.048771] [G loss: -2.617460]
[Epoch 20/200] [Batch 440/938] [D loss: -0.962639] [G loss: -2.550146]
[Epoch 20/200] [Batch 445/938] [D loss: -1.106331] [G loss: -2.646133]
[Epoch 20/200] [Batch 450/938] [D loss: -1.041970] [G loss: -2.618133]
[Epoch 20/200] [Batch 455/938] [D loss: -0.963351] [G loss: -3.253609]
[Epoch 20/200] [Batch 460/938] [D loss: -0.824771] [G loss: -3.273849]
[Epoch 20/200] [Batch 465/938] [D loss: -0.901267] [G loss: -2.406642]
[Epoch 20/200] [Batch 470/938] [D loss: -0.958502] [G loss: -2.031071]
[Epoch 20/200] [Batch 475/938] [D loss: -0.464755] [G loss: -2.620397]
[Epoch 20/200] [Batch 480/938] [D loss: -0.897697] [G loss: -2.581321]
[Epoch 20/200] [Batch 485/938] [D loss: -1.034294] [G loss: -2.476367]
[Epoch 20/200] [Batch 490/938] [D loss: -0.752021] [G loss: -2.519084]
[Epoch

KeyboardInterrupt: 